## Case 4

In this tutorial, we provide a demo on how to use FAIRSteam to generate ready-to-model datasets in both TensorFlow Dataset format and DataFrame format. Then, we provide 2 example models that take these modeling data as inputs and make either multi-class or binary classification. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import FAIRStream

## setup

In [2]:
# csv pool folder directory
csv_pool_path = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/csv_pool'
# current experiment working directory
work_dir = '/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_txp'

In [3]:
# initiate a FAIRStream object instance for BSI project
bsi_stream = FAIRStream.FAIRStream(work_dir)
# take a look at dictionaries in engineer's hands
#bsi_stream.engineer.csv_source_dict
bsi_stream.engineer.variable_dict

Success: Querier has initiated a csv source dictionary in:/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_txp/meta_data/csv_source_dict_demo.json
Success: Querier has initiated a variable dictionary in:/Users/jiaxingqiu/Documents/CAMA_projects/BSI/code/projects/case_txp/meta_data/variable_dict_demo.json
Unable to read sql source dictionary. Use Querier.update_sql_source_dict() to build one.


{'__uid': {'src_names': ['id', 'ID', 'subject_id', 'subject_id'],
  'label': 'unique subject id for current study',
  'unique_per_sbj': True},
 '__time': {'src_names': ['tsa', 'tsa', 'timeMinutes', 'timeMinutes'],
  'label': 'time since admission',
  'unit': 'minute'},
 '__anchor': {'src_names': ['True_positive', 'True positive'],
  'label': 'anchor for an episode',
  'unique_per_sbj': False,
  'factor': {'levels': {'__neg_nbc': ['0', '0.0', 'nan'],
    '__pos': ['1', '1.0']},
   'impute_per_sbj': {'nan_level': '__neg_nbc'}}},
 'y': {'output': True,
  'src_names': ['True_positive', 'True positive'],
  'label': 'Event outcome result',
  'unique_per_sbj': False,
  'factor': {'levels': {'nbc': ['nan'],
    'neg': ['0', '0.0'],
    'pos': ['1', '1.0']},
   'impute_per_sbj': {'nan_level': 'nbc'}}},
 'txp': {'input': True,
  'src_names': ['Confirmed Txp'],
  'label': 'Transplant recipient',
  'unique_per_sbj': True,
  'factor': {'levels': {'no': ['0', '0.0', 'nan'], 'yes': ['1', '1.0']},
   

## Define Episode

In [4]:
# define an episode (notice that the engineer now has new attributes)
bsi_stream.engineer.DefineEpisode(input_time_len=2*24*60, # using vital signs and labs 4 days prior to a culture 
                                  output_time_len=1, # predict one time unit into the future
                                  time_resolution=60, # aggregate minutely data to one row per hour 
                                  time_lag=0,  # no time lag between predictors and response
                                  anchor_gap=7*24*60) # the minimum distance between two episodes

Success! Engineer has updated attributes --- episode. 


In [5]:
print(bsi_stream.engineer.episode)

 
An episode is defined to 
--- use 2880 minute(s) long input variables 
--- predict 1 minute(s) response variables into the future
--- lag 0 minute(s) between predictors and responses
--- increase by every 60 minute(s)
--- last at most 10080 minute(s) long


## Build MVTS (multi-variable time series) data objects
- train_df_imputed, valid_df_imputed and test_df_imputed are dataframes
- train_tfds, valid_tfds and test_tfds are tensorflow datasets

In [ ]:
# Build MVTS dataframe or tfds  (notice that the engineer now has new attributes)
bsi_stream.engineer.BuildMVTS(csv_pool_path, 
                              nsbj = 500, # number of subjects / patients to sample from the pool 
                              replace=False, # sample with replacement or not 
                              valid_frac = 0.2, # fraction of number of subjects in validation dataset
                              test_frac = 0.1, # fraction of number of subjects in left-out test dataset
                              batch_size = 64, # batch size (usually 32,64,128..)
                              impute_input='median', # imputation on predictors
                              impute_output='median' )# imputation on response (no need in BSI project)

# please see the end of console 
# --- Success! Engineer has updated attributes --- train_df_imputed, valid_df_imputed and test_df_imputed. 
# --- Success! Engineer has updated attributes --- train_tfds, valid_tfds and test_tfds. 

### print all the attributes of the engineer

In [10]:
print(bsi_stream.engineer)


------------------ episode definition --------------

{'time_unit': 'minute', 'time_resolution': 60, 'input_time_len': 2880, 'output_time_len': 1, 'time_lag': 0, 'anchor_gap': 10080}

------------------ sampling info --------------

500---out of---5641---subjects are sampled from csv pool of size---26093

----------------------------- Engineer Attributes List ------------------------

['work_dir', 'meta_dir', 'variable_dict', 'csv_source_dict', 'sql_source_dict', 'input_vars', 'output_vars', 'episode', 'sample_info', 'mvts_df', 'mvts_tfds', 'train_df', 'valid_df', 'test_df', 'train_df_imputed', 'valid_df_imputed', 'test_df_imputed', 'train_tfds', 'valid_tfds', 'test_tfds', 'df_csv_fullname_ls', 'hist']

------------------------------------- Inputs  --------------------------------- 

['txp___no', 'txp___yes', 'age___vital', 'temp___vital', 'heart_rate___vital', 'resp_rate___vital', 'spo2___vital', 'x_hr_rr___vital', 's_hr___vital', 's_rr___vital', 's_so2___vital', 'systolic_blood_pres

In [11]:
bsi_stream.engineer.hist

{'episode': {'time_unit': 'minute',
  'time_resolution': 60,
  'input_time_len': 2880,
  'output_time_len': 1,
  'time_lag': 0,
  'anchor_gap': 10080},
 'split': {'stratify_by': None,
  'valid_frac': 0.2,
  'test_frac': 0.1,
  'byepisode': False,
  'first_neps': None},
 'imputation': {'x': 'median', 'y': 'median'},
 'data': {'x_cols': "['txp___no', 'txp___yes', 'age___vital', 'temp___vital', 'heart_rate___vital', 'resp_rate___vital', 'spo2___vital', 'x_hr_rr___vital', 's_hr___vital', 's_rr___vital', 's_so2___vital', 'systolic_blood_pressure___vital', 'diastolic_blood_pressure___vital', 'glucose___vital', 'bilirubin___vital', 'potassium___vital', 'albumin___vital', 'calcium___vital', 'sodium___vital', 'wbc___vital', 'phosphorus___vital', 'creatinine___vital', 'platelet_count___vital', 'alt___vital', 'alp___vital', 'ast___vital', 'pco2___vital', 'chloride___vital', 'troponin___vital', 'ptt___vital', 'lactate___vital', 'bun___vital', 'magnesium___vital']",
  'y_cols': "['y___nbc', 'y___po

## Multilabel classification -- by TFDS

In [23]:
train_tfds = bsi_stream.engineer.train_tfds
valid_tfds = bsi_stream.engineer.valid_tfds
input_shape = list(train_tfds.element_spec[0].shape)[1:3]
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC'),
    keras.metrics.AUC(name='AUPRC', curve='PR')
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.Flatten(input_shape=input_shape),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(3, activation='softmax'),
    keras.layers.Reshape([1, -1])
])
mdl.summary()
mdl.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=10)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-5)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-6)
his = mdl.fit(train_tfds, 
              epochs=50, 
              validation_data=valid_tfds, callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1584)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1584)              6336      
_________________________________________________________________
dense (Dense)                (None, 3)                 4755      
_________________________________________________________________
reshape (Reshape)            (None, 1, 3)              0         
Total params: 11,091
Trainable params: 7,923
Non-trainable params: 3,168
_________________________________________________________________
Epoch 1/50
7/7 [==============================] - 1s 76ms/step - loss: 1.4395 - AUROC: 0.5494 - AUPRC: 0.3690 - val_loss: 9.5567 - val_AUROC: 0.5553 - val_AUPRC: 0.3775
Epoch 2/50
7/7 [==============================] - 0s 27ms/step - loss: 0.9777 

7/7 [==============================] - 0s 30ms/step - loss: 0.6740 - AUROC: 0.8754 - AUPRC: 0.7964 - val_loss: 1.0058 - val_AUROC: 0.7482 - val_AUPRC: 0.5890
Epoch 31/50
7/7 [==============================] - 0s 26ms/step - loss: 0.6647 - AUROC: 0.8797 - AUPRC: 0.8017 - val_loss: 0.9971 - val_AUROC: 0.7488 - val_AUPRC: 0.5879
Epoch 32/50
7/7 [==============================] - 0s 29ms/step - loss: 0.6602 - AUROC: 0.8818 - AUPRC: 0.8044 - val_loss: 0.9891 - val_AUROC: 0.7496 - val_AUPRC: 0.5882
Epoch 33/50
7/7 [==============================] - 0s 28ms/step - loss: 0.6596 - AUROC: 0.8828 - AUPRC: 0.8098 - val_loss: 0.9828 - val_AUROC: 0.7507 - val_AUPRC: 0.5887
Epoch 34/50
7/7 [==============================] - 0s 27ms/step - loss: 0.6816 - AUROC: 0.8692 - AUPRC: 0.7806 - val_loss: 0.9763 - val_AUROC: 0.7541 - val_AUPRC: 0.5910
Epoch 35/50
7/7 [==============================] - 0s 32ms/step - loss: 0.6737 - AUROC: 0.8742 - AUPRC: 0.7897 - val_loss: 0.9705 - val_AUROC: 0.7599 - val_AUPRC:

## Binary classification -- by DataFrame

In [43]:
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = bsi_stream.engineer.ExtractXY()

X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')

Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[2])
Y_valid = Y_valid.reshape(Y_valid.shape[0], Y_valid.shape[2])
Y_test = Y_test.reshape(Y_test.shape[0], Y_test.shape[2])
Y_train = Y_train[:,1]
Y_valid = Y_valid[:,1]
Y_test = Y_test[:,1]
Y_train = Y_train.reshape(Y_train.shape[0], 1)
Y_valid = Y_valid.reshape(Y_valid.shape[0], 1)
Y_test = Y_test.reshape(Y_test.shape[0], 1)

X_all = np.concatenate((X_train, X_valid, X_test), axis=0)
Y_all = np.concatenate((Y_train, Y_valid, Y_test), axis=0)

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)
print("X_valid shape", X_valid.shape)
print("Y_valid shape", Y_valid.shape)
print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)
print("X_all shape", X_all.shape)
print("Y_all shape", Y_all.shape)

X_train shape (428, 48, 33)
Y_train shape (428, 1)
X_valid shape (113, 48, 33)
Y_valid shape (113, 1)
X_test shape (65, 48, 33)
Y_test shape (65, 1)
X_all shape (606, 48, 33)
Y_all shape (606, 1)


In [44]:
myMetrics = [
    keras.metrics.AUC(name='AUROC', curve='ROC'),
    keras.metrics.AUC(name='AUPRC', curve='PR')
]

keras.backend.clear_session()
mdl = keras.models.Sequential([
    keras.layers.LSTM(16, return_sequences=True, input_shape=list(X_train.shape)[1:3]),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(16, return_sequences=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='sigmoid')
])
mdl.summary()
mdl.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics = myMetrics)
callback = EarlyStopping(monitor='val_loss', patience=10)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-4)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-5)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])
keras.backend.set_value(mdl.optimizer.learning_rate, 1e-6)
his = mdl.fit(X_train, Y_train, 
              epochs=50, 
              validation_data=(X_valid,Y_valid), callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 48, 16)            3200      
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 16)            64        
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                2112      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 5,457
Trainable params: 5,393
Non-trainable params: 64
_________________________________________________________________
Epoch 1/50
14/14 [==============================] - 4s 85ms/step - loss: 0.7603 - AUROC: 0.5149 - AUPRC: 0.2292 - val_l

Epoch 43/50
14/14 [==============================] - 0s 25ms/step - loss: 0.3627 - AUROC: 0.7877 - AUPRC: 0.5853 - val_loss: 0.3094 - val_AUROC: 0.6823 - val_AUPRC: 0.1738
Epoch 44/50
14/14 [==============================] - 0s 26ms/step - loss: 0.3029 - AUROC: 0.8178 - AUPRC: 0.6090 - val_loss: 0.3090 - val_AUROC: 0.6907 - val_AUPRC: 0.1879
Epoch 45/50
14/14 [==============================] - 0s 24ms/step - loss: 0.3586 - AUROC: 0.7733 - AUPRC: 0.5790 - val_loss: 0.3117 - val_AUROC: 0.6720 - val_AUPRC: 0.1625
Epoch 46/50
14/14 [==============================] - 0s 26ms/step - loss: 0.3650 - AUROC: 0.7300 - AUPRC: 0.4555 - val_loss: 0.3431 - val_AUROC: 0.5704 - val_AUPRC: 0.1307
Epoch 47/50
14/14 [==============================] - 0s 25ms/step - loss: 0.3506 - AUROC: 0.7351 - AUPRC: 0.4754 - val_loss: 0.3210 - val_AUROC: 0.6961 - val_AUPRC: 0.1681
Epoch 48/50
14/14 [==============================] - 0s 25ms/step - loss: 0.3194 - AUROC: 0.7533 - AUPRC: 0.5896 - val_loss: 0.3267 - val_AU